<a href="https://colab.research.google.com/github/drdww/OPIM5641/blob/main/Module5/M5_1/2_Assignment_SwimTeam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Network Problems: Assignment (Swim Team)

**OPIM 5641: Business Decision Modeling - University of Connecticut**

The inequalities in the textbook could probably be written as equalities (==), but it still works. Great example that generalizes/motivates many other types of machine scheduling problems.

* Please refer to Powell Chapter 10 for more details and examples.
* Pyomo Cookbook (related example): https://github.com/jckantor/ND-Pyomo-Cookbook/blob/master/notebooks/03.00-Assignment-Problems.ipynb

-------------------------------------------------------------------------

In [ ]:
# import modules

%matplotlib inline
from pylab import *

import shutil
import sys
import os.path

if not shutil.which("pyomo"):
    !pip install -q pyomo
    assert(shutil.which("pyomo"))

if not (shutil.which("cbc") or os.path.isfile("cbc")):
    if "google.colab" in sys.modules:
        !apt-get install -y -qq coinor-cbc
    else:
        try:
            !conda install -c conda-forge coincbc 
        except:
            pass

assert(shutil.which("cbc") or os.path.isfile("cbc"))

from pyomo.environ import *
# ensure you have cbc installed
!apt-get install -y -qq coinor-cbc

     |████████████████████████████████| 9.4MB 4.4MB/s 
     |████████████████████████████████| 256kB 43.6MB/s 
     |████████████████████████████████| 51kB 5.7MB/s 
     |████████████████████████████████| 163kB 52.6MB/s 
Selecting previously unselected package coinor-libcoinutils3v5.
(Reading database ... 144618 files and directories currently installed.)
Preparing to unpack .../0-coinor-libcoinutils3v5_2.10.14+repack1-1_amd64.deb ...
Unpacking coinor-libcoinutils3v5 (2.10.14+repack1-1) ...
Selecting previously unselected package coinor-libosi1v5.
Preparing to unpack .../1-coinor-libosi1v5_0.107.9+repack1-1_amd64.deb ...
Unpacking coinor-libosi1v5 (0.107.9+repack1-1) ...
Selecting previously unselected package coinor-libclp1.
Preparing to unpack .../2-coinor-libclp1_1.16.11+repack1-1_amd64.deb ...
Unpacking coinor-libclp1 (1.16.11+repack1-1) ...
Selecting previously unselected package coinor-libcgl1.
Preparing to unpack .../3-coinor-libcgl1_0.59.10+repack1-1_amd64.deb ...
Unpacking coi

# Example
Coach Kemppel is the coach of the Buchanan Swim Club’s co-ed team. Her team competes against other swim clubs, and a perennial question for the coach is how to organize the medley relay team. The medley relay requires four swimmers to each swim a different stroke: butterfly, breaststroke, backstroke, and freestyle. The relay is the final event in the competitions, and the outcome of the swim meet often depends on the performance of the relay team. During practice, Coach Kemppel has asked each of her top four swimmers to try each of the four strokes, and she has tracked their times (in seconds), as shown in the following table:

Person | Butterfly (1) | Breaststroke (2) | Backstroke (3) | Freestyle (4) | 
--- | --- | --- | --- | --- | 
Todd | 38 | 75 | 44 | 27 | 
Betsy | 34 | 76 | 43 | 25 | 
Lee | 41 | 71 | 41 | 26 | 
Carly | 33 | 80 | 45 | 30 | 

With this information, Coach Kemppel is ready to assign swimmers to strokes in the relay race, but she can see that a lot of combinations are possible.

For an algebraic statement of our model, we define our decision variables as the possible swimmer-stroke combinations, T1, T2, . . . , C4, where the letter refers to the swimmer’s name and the number refers to the stroke (1 for Butterfly, etc.). Our objective function (denoted z) is the total time for an assignment, which can be expressed as the sum of sixteen products. Each term in this sum is a parameter multiplied by a decision variable:

**Objective Function:** $\min(\text{Time}) = 38_{T1} + 75_{T2} + 44_{T3} + 27_{T4} + 34_{B1} + 76_{B2} + 43_{B3} + 25_{B4} + 41_{L1} + 71_{L2} + 41_{L3} + 26_{L4} + 33_{C1} + 80_{C2} + 45_{C3} + 30_{C4}$

Subject to constraints:

*(that each person can only do one stroke!)*

$T1 + T2 + T3 + T4 \leq 1$

$B1 + B2 + B3 + B4 \leq1$

$L1 + L2 + L3 + L4 \leq1$

$C1 + C2 + C3 + C4 \leq1$

*(that each stroke needs one person!)*

$T1 + B1 + L1 + C1 \geq 1$

$T2 + B2 + L2 + C2 \geq 1$

$T3 + B3 + L3 + C3 \geq 1$

$T4 + B4 + L4 + C4 \geq 1$


# Solved Like Powell

In [ ]:
# declare the model
model = ConcreteModel()

# write your variables
# TODD
model.T1 = Var(domain=NonNegativeIntegers)
model.T2 = Var(domain=NonNegativeIntegers)
model.T3 = Var(domain=NonNegativeIntegers)
model.T4 = Var(domain=NonNegativeIntegers)
# BETSY
model.B1 = Var(domain=NonNegativeIntegers)
model.B2 = Var(domain=NonNegativeIntegers)
model.B3 = Var(domain=NonNegativeIntegers)
model.B4 = Var(domain=NonNegativeIntegers)
# LEE
model.L1 = Var(domain=NonNegativeIntegers)
model.L2 = Var(domain=NonNegativeIntegers)
model.L3 = Var(domain=NonNegativeIntegers)
model.L4 = Var(domain=NonNegativeIntegers)
# CARLY
model.C1 = Var(domain=NonNegativeIntegers)
model.C2 = Var(domain=NonNegativeIntegers)
model.C3 = Var(domain=NonNegativeIntegers)
model.C4 = Var(domain=NonNegativeIntegers)

In [ ]:
# write the objective function
model.OBJ = Objective(expr = 38*model.T1 + 75*model.T2 + 44*model.T3 + 27*model.T4 +
                            34*model.B1 + 76*model.B2 + 43*model.B3 + 25*model.B4 + 
                            41*model.L1 + 71*model.L2 + 41*model.L3 + 26*model.L4 +
                            33*model.C1 + 80*model.C2 + 45*model.C3 + 30*model.C4,
                      sense=minimize)

In [ ]:
# write the constraints

# 'each swimmer be assigned to at most one stroke'
model.OneStoke_T = Constraint(expr = model.T1 + model.T2 + model.T3 + model.T4 == 1)
model.OneStoke_B = Constraint(expr = model.B1 + model.B2 + model.B3 + model.B4 == 1)
model.OneStoke_L = Constraint(expr = model.L1 + model.L2 + model.L3 + model.L4 == 1)
model.OneStoke_C = Constraint(expr = model.C1 + model.C2 + model.C3 + model.C4 == 1)

# 'each stroke be assigned one swimmer' (note equality constraint, ==)
model.Do_Stroke1 = Constraint(expr = model.T1 + model.B1 + model.L1 + model.C1 == 1)
model.Do_Stroke2 = Constraint(expr = model.T2 + model.B2 + model.L2 + model.C2 == 1)
model.Do_Stroke3 = Constraint(expr = model.T3 + model.B3 + model.L3 + model.C3 == 1)
model.Do_Stroke4 = Constraint(expr = model.T4 + model.B4 + model.L4 + model.C4 == 1)

Pretty print to inspect.

In [ ]:
model.pprint()

16 Var Declarations
    B1 : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :  None :  None : False :  True : NonNegativeIntegers
    B2 : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :  None :  None : False :  True : NonNegativeIntegers
    B3 : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :  None :  None : False :  True : NonNegativeIntegers
    B4 : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :  None :  None : False :  True : NonNegativeIntegers
    C1 : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :  None :  None : False :  True : NonNegativeIntegers
    C2 : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :  None :  None : False :  True : NonNegativ

Now solve it!

In [ ]:
# Solve the model
SolverFactory('cbc', executable='/usr/bin/cbc').solve(model).write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 173.0
  Upper bound: 173.0
  Number of objectives: 1
  Number of constraints: 8
  Number of variables: 16
  Number of binary variables: 0
  Number of integer variables: 12
  Number of nonzeros: 16
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  User time: -1.0
  System time: 0.0
  Wallclock time: 0.0
  Termination condition: optimal
  Termination message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Statistics: 
    Branch and bound: 
 

Show the results.

In [ ]:
# show the results
print('Total time:',model.OBJ())

Total time: 173.0


In [ ]:
# nice way to show assignments?
print("T:",model.T1(), model.T2 (), model.T3(), model.T4())
print("L:",model.L1(), model.L2 (), model.L3(), model.L4())
print("C:",model.C1(), model.C2 (), model.C3(), model.C4())
print("B:",model.B1(), model.B2 (), model.B3(), model.B4())

# you could turn this into a pandas dataframe if you wanted to (left to students!)

T: 0.0 0.0 1.0 0.0
L: 0.0 1.0 0.0 0.0
C: 1.0 0.0 0.0 0.0
B: 0.0 0.0 0.0 1.0


In [ ]:
# try it on your own
T = [model.T1(), model.T2 (), model.T3(), model.T4()]
L = [model.L1(), model.L2 (), model.L3(), model.L4()]
C = [model.C1(), model.C2 (), model.C3(), model.C4()]
B = [model.B1(), model.B2 (), model.B3(), model.B4()]

In [ ]:
# almost there! just need to rename rownames and column names... could color code?!
import pandas as pd
df = pd.DataFrame([T, L, C, B])
df.rename({0:'Butterfly', 1:'Breaststroke', 2:'Backstroke', 3:'Freestyle'}, axis=1, inplace=True) # axis=1 means columns
df.rename({0:'Todd', 1:'Lee', 2:'Carly', 3:'Betsy'}, axis=0, inplace=True) # axis=0 means rows
df
df

,Butterfly,Breaststroke,Backstroke,Freestyle
Todd,0.0,0.0,1.0,0.0
Lee,0.0,1.0,0.0,0.0
Carly,1.0,0.0,0.0,0.0
Betsy,0.0,0.0,0.0,1.0


# On Your Own
Students may try different times (within +/- 5 seconds) for each stroke, one at a time. You can 'tinker' and update cells one at a time, or you can use a for loop.

If you use the loop, students can also store all results and analyze the outputs probabilistically and give a 90% chance of achieving a particular time or better... sports betting, anyone?! 

In [ ]:
# students may try to show:
# 1) how the optimal solution varies as Todd’s time in the backstroke increases from 44 to 46 seconds
# 2) Carly’s time in the backstroke increases from 33 to 38 seconds :

# Solved According To Pyomo Cookbook

In [ ]:
Time = {
    ('T','1'): 38,
    ('T','2'): 75,
    ('T','3'): 44,
    ('T','4'): 27,
    ('B','1'): 34,
    ('B','2'): 76,
    ('B','3'): 43,
    ('B','4'): 25,
    ('L','1'): 41,
    ('L','2'): 71,
    ('L','3'): 41,
    ('L','4'): 26,
    ('C','1'): 33,
    ('C','2'): 80,
    ('C','3'): 45,
    ('C','4'): 30
}

assignments = list(Time.keys())

machines = ('T','B','L','C')
jobs = ('1','2','3','4')

In [ ]:
def create_bounds(model, i, j):
   return (0,1)

# declare the model
model = ConcreteModel()

# Create variables
model.x = Var(machines, jobs, domain = NonNegativeReals, bounds=create_bounds)

# Constraints
model.machine_constraints = ConstraintList()

# At most one job per machine - for loop is much more compact notation
for machine in machines:
  assign_expr = 0
  for job in jobs:
    assign_expr += model.x[machine,job]
  model.machine_constraints.add(assign_expr <= 1)


# Exactly one machine per job - for loop (again) is much more compact!
model.job_constraints = ConstraintList()
for job in jobs:
  assign_expr = 0
  for machine in machines:
    assign_expr += model.x[machine,job]
  model.job_constraints.add(assign_expr == 1)


# Objective - nice compact way (again!)
obj_expr = 0.0
for time in Time:
  print(time,Time[time])
  obj_expr += Time[time]*model.x[time]

model.total_time = Objective(
    expr = obj_expr, 
    sense = minimize)

model.pprint()

('T', '1') 38
('T', '2') 75
('T', '3') 44
('T', '4') 27
('B', '1') 34
('B', '2') 76
('B', '3') 43
('B', '4') 25
('L', '1') 41
('L', '2') 71
('L', '3') 41
('L', '4') 26
('C', '1') 33
('C', '2') 80
('C', '3') 45
('C', '4') 30
5 Set Declarations
    job_constraints_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    4 : {1, 2, 3, 4}
    machine_constraints_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    4 : {1, 2, 3, 4}
    x_index : Size=1, Index=None, Ordered=True
        Key  : Dimen : Domain              : Size : Members
        None :     2 : x_index_0*x_index_1 :   16 : {('T', '1'), ('T', '2'), ('T', '3'), ('T', '4'), ('B', '1'), ('B', '2'), ('B', '3'), ('B', '4'), ('L', '1'), ('L', '2'), ('L', '3'), ('L', '4'), ('C', '1'), ('C', '2'), ('C', '3'), ('C', '4')}
    x_index_0 : Size=1, Index=None, Ordered=Insertion
        Key  : Di

In [ ]:
# Solve the model
SolverFactory('cbc', executable='/usr/bin/cbc').solve(model).write()

# show the results
print('Total time:',model.total_time())

print("List of assigments")
for assignment in assignments:
  if 0 < model.x[assignment]():
    print(assignment)

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 173.0
  Upper bound: 173.0
  Number of objectives: 1
  Number of constraints: 9
  Number of variables: 17
  Number of nonzeros: 16
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  User time: -1.0
  System time: 0.0
  Wallclock time: 0.0
  Termination condition: optimal
  Termination message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: None
      Number of created s